In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Problem Staement**

Fatalities due to traffic delays of emergency vehicles such as ambulance & fire brigade is a huge problem. In daily life, we often see that emergency vehicles face difficulty in passing through traffic. So differentiating a vehicle into an emergency and non emergency category can be an important component in traffic monitoring as well as self drive car systems as reaching on time to their destination is critical for these services.

In this problem, you will be working on classifying vehicle images as either belonging to the emergency vehicle or non-emergency vehicle category. For the same, you are provided with the train and the test dataset. Emergency vehicles usually includes police cars, ambulance and fire brigades.

**Data Description**

train.zip: contains 2 csvs and 1 folder containing image data

train.csv – [‘image_names’, ‘emergency_or_not’] contains the image name and correct class for 1646 (70%) train images

images – contains 2352 images for both train and test sets

test.csv: [‘image_names’] contains just the image names for the 706 (30%) test images

sample_submission.csv: [‘image_names’,’emergency_or_not­’] contains the exact format for a valid submission (1 - For Emergency Vehicle, 0 - For Non Emergency Vehicle)

**Evaluation Metric**

The evaluation metric for this competition is Accuracy.

In [ ]:
import cv2 as cv
import pandas as pd 
import numpy as np 

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout,AveragePooling2D,BatchNormalization,Dense,Dropout
import matplotlib.pyplot as plt
from PIL import Image

from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16,ResNet50,DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras import models
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import *

import random
random.seed(42)

**Preparing the test and train generators**

In [ ]:
train_datagen=ImageDataGenerator(horizontal_flip=True,preprocessing_function=preprocess_input,
                                 width_shift_range=0.2,
                                 zoom_range=0.2,
                                height_shift_range=0.2,
                                 rotation_range=30)
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)


traindir = "../input/janatahack-computer-vision/data/train"
testdir = "../input/janatahack-computer-vision/data/val"

train_generator=train_datagen.flow_from_directory(
    traindir,
    target_size =(224,224),
    class_mode="categorical",
    batch_size=64
)

test_generator=test_datagen.flow_from_directory(
testdir,
target_size =(224,224),
class_mode="categorical",
batch_size=64
)

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)



print(train_generator.classes)
print(train_generator.class_indices)
print(class_weights)

**Building the model Updating the weights trained from imagenet**

**Include top = False means whether to include the fully connected layer at the top of the network, we don't require it so we have set it to False.**

In [ ]:
resnet = DenseNet121(weights="imagenet",include_top=False,input_shape=(224,224,3))

model = models.Sequential()
model.add(resnet)
model.add(Flatten())
model.add(Dense(2,activation="softmax"))
model.summary()

**Fitting the model using SGD optimizer**

**Giving more weight to class 1 as it is highly imbalanced**

In [ ]:
sgd = optimizers.Nadam(lr=0.00001)
checkpoint = ModelCheckpoint('./models/model-{epoch:03d}.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')  
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

history = model.fit_generator( train_generator,
                              epochs=5, 
                              validation_data=test_generator,
                              class_weight={0:1.22503962,1:0.84480874},callbacks=[checkpoint])

**Saving the model**

In [ ]:
model.save("model.h5")

**Loading the model**

In [ ]:
model=load_model("./model.h5")

**Predictions on test data**

In [ ]:
testdir = "../input/janatahack-computer-vision/data/test"

images=[]
names=[]
for i in os.listdir():
    if i[-4:]!=".jpg":
        continue
    img=cv.imread(i)
    img=preprocess_input(img)
    images.append(img)
    names.append(i)

In [ ]:
data=np.array(images)
data.shape

In [ ]:
preds=model.predict(data)

**Considering a cut-off threshold of 0.5 for predictions, all predictions greater than 0.5 are mapped to class 1 and all predictions lesser than 0.5 are mapped to class 0**

In [ ]:
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

In [ ]:
print(preds)

**Finding the class with the largest predicted probability using argmax**

In [ ]:
max_pred = np.argmax(preds, axis=1)
max_pred

**Subtracting 1 to get the optimum class**

In [ ]:
final_output = abs(1 - max_pred)

In [ ]:
result = pd.DataFrame({"image_names":names,"emergency_or_not": final_output})
result.head(10)

**Submitting the predictions**

In [ ]:
result.to_csv('/kaggle/working/submission.csv' , index = None)